# Earthquake Prediction using Swarm AI and Multi-Agent Reinforcement Learning

This notebook implements a sophisticated earthquake prediction system using:
- Multi-Agent Reinforcement Learning (MARL)
- Swarm Intelligence Algorithms
- Deep Learning for time series prediction

Author: abhinavuser
Date: 2025-05-09

In [ ]:
# Install required packages
!pip install -q pettingzoo==1.24.1
!pip install -q stable-baselines3==2.1.0
!pip install -q geopandas==0.14.1 folium==0.15.1
!pip install -q torch==2.1.0 torchvision==0.16.0
!pip install -q gymnasium==0.29.1

# Import necessary libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import folium
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gymnasium as gym
from pettingzoo import ParallelEnv
from pettingzoo.utils import parallel_to_aec
from stable_baselines3 import PPO

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Enable GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
class EarthquakeDataProcessor:
    def __init__(self, start_date=None, end_date=None):
        self.start_date = start_date or (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
        self.end_date = end_date or datetime.now().strftime('%Y-%m-%d')
        self.base_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
        
    def load_data(self):
        """Load earthquake data from USGS API"""
        params = {
            'format': 'csv',
            'starttime': self.start_date,
            'endtime': self.end_date,
            'minmagnitude': 2.5
        }
        
        self.df = pd.read_csv(self.base_url, params=params)
        print(f'Loaded {len(self.df)} earthquake records')
        return self.df
    
    def preprocess_data(self):
        """Preprocess earthquake data for model input"""
        # Convert time to datetime
        self.df['datetime'] = pd.to_datetime(self.df['time'])
        
        # Extract temporal features
        self.df['hour'] = self.df['datetime'].dt.hour
        self.df['day'] = self.df['datetime'].dt.day
        self.df['month'] = self.df['datetime'].dt.month
        self.df['year'] = self.df['datetime'].dt.year
        self.df['dayofweek'] = self.df['datetime'].dt.dayofweek
        
        # Calculate additional features
        self.df['depth_km'] = self.df['depth'].abs()
        self.df['energy'] = 10 ** (1.5 * self.df['mag'])
        
        # Normalize features
        features = ['latitude', 'longitude', 'depth_km', 'mag', 'energy']
        self.scaler = StandardScaler()
        self.df[features] = self.scaler.fit_transform(self.df[features])
        
        return self.df

        def create_sequences(self, seq_length=24):
            """Create sequences for time series prediction"""
            features = ['latitude', 'longitude', 'depth_km', 'mag', 'energy',
                       'hour', 'day', 'month', 'year', 'dayofweek']
            
            sequences = []
            targets = []
            
            for i in range(len(self.df) - seq_length):
                seq = self.df[features].iloc[i:i+seq_length].values
                target = self.df[['latitude', 'longitude', 'mag']].iloc[i+seq_length].values
                sequences.append(seq)
                targets.append(target)
            
            return np.array(sequences), np.array(targets)

In [ ]:
class EarthquakeEnvironment(ParallelEnv):
    def __init__(self, num_agents=5, grid_size=50, max_steps=1000):
        super().__init__()
        self.num_agents = num_agents
        self.grid_size = grid_size
        self.max_steps = max_steps
        self.current_step = 0
        
        # Define agents
        self.possible_agents = [f'agent_{i}' for i in range(num_agents)]
        self.agents = self.possible_agents.copy()
        
        # Define observation and action spaces
        self.observation_spaces = {agent: gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(10,), dtype=np.float32
        ) for agent in self.possible_agents}
        
        self.action_spaces = {agent: gym.spaces.Box(
            low=-1, high=1, shape=(4,), dtype=np.float32
        ) for agent in self.possible_agents}
    
        def reset(self, seed=None, options=None):
            """Reset environment to initial state"""
            self.agents = self.possible_agents.copy()
            self.current_step = 0
            
            observations = {agent: self._get_observation() 
                                        for agent in self.agents}
            infos = {agent: {} for agent in self.agents}
            
            return observations, infos
        
        def step(self, actions):
            """Execute environment step"""
            # Execute actions and get rewards
            rewards = {agent: self._calculate_reward(actions[agent]) 
                      for agent in self.agents}
            
            # Get new observations
            observations = {agent: self._get_observation() 
                          for agent in self.agents}
            
            # Check termination conditions
            self.current_step += 1
            terminated = {agent: self.current_step >= self.max_steps 
                         for agent in self.agents}
            truncated = {agent: False for agent in self.agents}
            infos = {agent: {} for agent in self.agents}
            
            return observations, rewards, terminated, truncated, infos
        
        def _get_observation(self):
            """Generate observation for an agent"""
            return np.random.normal(0, 1, (10,)).astype(np.float32)
        
        def _calculate_reward(self, action):
            """Calculate reward for an action"""
            # Implement your reward function here
            return float(np.sum(action ** 2))

In [ ]:
class ParticleSwarmOptimization:
    def __init__(self, num_particles, dimensions, bounds, w=0.7, c1=1.5, c2=1.5):
        self.num_particles = num_particles
        self.dimensions = dimensions
        self.bounds = bounds
        self.w = w  # inertia weight
        self.c1 = c1  # cognitive weight
        self.c2 = c2  # social weight
        
        # Initialize particles
        self.particles = self._initialize_particles()
        self.global_best_position = None
        self.global_best_score = float('inf')
    
        def _initialize_particles(self):
            """Initialize particle positions and velocities"""
            particles = []
            for _ in range(self.num_particles):
                position = np.random.uniform(
                    self.bounds[0], self.bounds[1], self.dimensions
                )
                velocity = np.zeros(self.dimensions)
                particles.append({
                    'position': position,
                    'velocity': velocity,
                    'best_position': position.copy(),
                    'best_score': float('inf')
                })
            return particles
        
        def optimize(self, objective_function, max_iterations=100):
            """Run PSO optimization"""
            for iteration in range(max_iterations):
                for particle in self.particles:
                    # Evaluate current position
                    score = objective_function(particle['position'])
                    
                    # Update personal best
                    if score < particle['best_score']:
                        particle['best_score'] = score
                        particle['best_position'] = particle['position'].copy()
                    
                    # Update global best
                    if score < self.global_best_score:
                        self.global_best_score = score
                        self.global_best_position = particle['position'].copy()
                    
                    # Update velocity and position
                    r1, r2 = np.random.rand(2)
                    particle['velocity'] = (
                        self.w * particle['velocity'] +
                        self.c1 * r1 * (particle['best_position'] - particle['position']) +
                        self.c2 * r2 * (self.global_best_position - particle['position'])
                    )
                    
                    # Update position
                    particle['position'] += particle['velocity']
                    particle['position'] = np.clip(
                        particle['position'], self.bounds[0], self.bounds[1]
                    )
                
                if iteration % 10 == 0:
                    print(f'Iteration {iteration}, Best Score: {self.global_best_score:.6f}')
            
            return self.global_best_position, self.global_best_score

In [ ]:
class EarthquakePredictionModel(nn.Module):
    def __init__(self, input_size=10, hidden_size=64, num_layers=2, output_size=3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, output_size)
        )
    
        def forward(self, x):
            lstm_out, _ = self.lstm(x)
            predictions = self.fc(lstm_out[:, -1, :])
            return predictions

        class EarthquakeDataset(Dataset):
            def __init__(self, sequences, targets):
                self.sequences = torch.FloatTensor(sequences)
                self.targets = torch.FloatTensor(targets)
            
            def __len__(self):
                return len(self.sequences)
            
            def __getitem__(self, idx):
                return self.sequences[idx], self.targets[idx]

In [ ]:
def train_model(model, train_loader, val_loader, epochs=100, learning_rate=0.001):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
    
    best_val_loss = float('inf')
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                val_loss += criterion(output, target).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
        
        print(f'Epoch: {epoch+1}/{epochs}')
        print(f'Training Loss: {train_loss:.6f}')
        print(f'Validation Loss: {val_loss:.6f}')
        print('-' * 50)
    
    return model

In [ ]:
def main():
    # Initialize data processor and load data
    processor = EarthquakeDataProcessor()
    data = processor.load_data()
    processed_data = processor.preprocess_data()
    
    # Create sequences for training
    sequences, targets = processor.create_sequences(seq_length=24)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        sequences, targets, test_size=0.2, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    # Create datasets and dataloaders
    train_dataset = EarthquakeDataset(X_train, y_train)
    val_dataset = EarthquakeDataset(X_val, y_val)
    test_dataset = EarthquakeDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    test_loader = DataLoader(test_dataset, batch_size=32)
    
    # Initialize model
    model = EarthquakePredictionModel(
        input_size=10,
        hidden_size=64,
        num_layers=2,
        output_size=3
    )
    
    # Train model
    trained_model = train_model(
        model,
        train_loader,
        val_loader,
        epochs=100,
        learning_rate=0.001
    )
    
    # Initialize MARL environment
    env = EarthquakeEnvironment(num_agents=5)
    
    # Initialize PSO optimizer
    pso = ParticleSwarmOptimization(
        num_particles=20,
        dimensions=model.count_parameters(),
        bounds=(-1, 1)
    )
    
    print('Training complete!')

        if __name__ == '__main__':
            main()